# Create the dataframe

In [1]:
import requests 
import pandas as pd 
import numpy as np
import random 


#!pip install geopy
from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 

from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
#pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup

response = requests.get(url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
link = soup.find_all('tr')
column_names = ['Postal Code','Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
len(link)


185

In [6]:
for l in link:
    if len(l)<3:
        continue 
    Postal =str(l.contents[1].text)[0:3]
    borough = str(l.contents[3].text)[0:-1]
    neighborhood = str(l.contents[5].text)[0:-1]
 

    if Postal[0]=="M":
        neighborhoods = neighborhoods.append({'Postal Code': Postal,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood},
                                           ignore_index=True)
    

In [7]:
neighborhoods

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:

neighborhoods = neighborhoods[neighborhoods['Borough'] != "Not assigned"].reset_index(drop=True)
print("We have {} Postal Code".format(neighborhoods.shape[0]))

We have 103 Postal Code


# Create the dataframe with latitude and longitude

In [31]:
neighborhoods.iloc[0]['Postal Code']
coords = pd.read_csv("Geospatial_Coordinates.csv")
coords.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [36]:

#neighborhoods = pd.merge(neighborhoods, coords, on="Postal Code")
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Explore and cluster the neighborhoods in Toronto

## Create a map of Toronto with neighborhoods

In [34]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [94]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [43]:
NorthYork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.shape

(24, 5)

### Explore Downtown North York

In [53]:
CLIENT_ID = 'SRCMSC5JQ35QCVDCCK4IKKB1ALW3NVBDGASRH3DQOM0LERBX'
CLIENT_SECRET = 'KWFZACWWH2AOZTVPXPN4YZ4APXO0JNYOOSOH0F4FFIZP1WIS' 
ACCESS_TOKEN = 'KEL0PP30CQBOUGQPPSBVU22PUR2NF3YL3VEQV3LXAXFB51I1'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SRCMSC5JQ35QCVDCCK4IKKB1ALW3NVBDGASRH3DQOM0LERBX
CLIENT_SECRET:KWFZACWWH2AOZTVPXPN4YZ4APXO0JNYOOSOH0F4FFIZP1WIS


In [54]:
neighborhood_latitude = NorthYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NorthYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NorthYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [59]:
search_query= 'venues'
radius =1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [77]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

In [78]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b8991cbf964a520814232e3',
   'name': "Allwyn's Bakery",
   'location': {'address': '81 Underhill drive',
    'lat': 43.75984035203157,
    'lng': -79.32471879917513,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.75984035203157,
      'lng': -79.32471879917513}],
    'distance': 833,
    'postalCode': 'M3A 1Z5',
    'cc': 'CA',
    'neighborhood': 'Parkwoods - Donalda',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['81 Underhill drive',
     'Toronto ON M3A 1Z5',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d144941735',
     'name': 'Caribbean Restaurant',
     'pluralName': 'Caribbean Restaurants',
     'shortName': 'Caribbean',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
      'suffix': '.png'},
     'primary': 

In [88]:
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

/Users/zhangmengxuan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.venuePage.id
0,e-0-4b8991cbf964a520814232e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b8991cbf964a520814232e3,Allwyn's Bakery,81 Underhill drive,43.759840,-79.324719,"[{'label': 'display', 'lat': 43.75984035203157...",833,...,Parkwoods - Donalda,Toronto,ON,Canada,"[81 Underhill drive, Toronto ON M3A 1Z5, Canada]","[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",0,[],NaN,NaN
1,e-0-57e286f2498e43d84d92d34a-1,0,"[{'summary': 'This spot is popular', 'type': '...",57e286f2498e43d84d92d34a,Tim Hortons,215 Brookbanks,43.760668,-79.326368,"[{'label': 'display', 'lat': 43.76066827030228...",866,...,NaN,Toronto,ON,Canada,"[215 Brookbanks (York Miils Rd), Toronto ON M3...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],York Miils Rd,NaN
2,e-0-4e8d9dcdd5fbbbb6b3003c7b-2,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,...,NaN,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,600917367
3,e-0-4bafa285f964a5203a123ce3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4bafa285f964a5203a123ce3,Bruno's valu-mart,83 Underhill,43.746143,-79.324630,"[{'label': 'display', 'lat': 43.746143, 'lng':...",889,...,NaN,Don Mills,ON,Canada,"[83 Underhill (at Donwood Plaza), Don Mills ON...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[],at Donwood Plaza,NaN
4,e-0-4b845f3ff964a520733131e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b845f3ff964a520733131e3,High Street Fish & Chips,55 Underhill Dr.,43.745260,-79.324949,"[{'label': 'display', 'lat': 43.74525988387223...",967,...,NaN,Toronto,ON,Canada,"[55 Underhill Dr. (at Donwood Plaza), Toronto ...","[{'id': '4edd64a0c7ddd24ca188df1a', 'name': 'F...",0,[],at Donwood Plaza,NaN
5,e-0-58a8dcaa6119f47b9a94dc05-5,0,"[{'summary': 'This spot is popular', 'type': '...",58a8dcaa6119f47b9a94dc05,A&W,1277 York Mills Road,43.760643,-79.326865,"[{'label': 'display', 'lat': 43.76064307616131...",852,...,NaN,Toronto,ON,Canada,"[1277 York Mills Road, Toronto ON M3A 1Z5, Can...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[],NaN,NaN
6,e-0-4c422e48e26920a1a4ad5fe7-6,0,"[{'summary': 'This spot is popular', 'type': '...",4c422e48e26920a1a4ad5fe7,Shoppers Drug Mart,1277 York Mills Rd,43.760857,-79.324961,"[{'label': 'display', 'lat': 43.76085733239677...",926,...,NaN,Toronto,ON,Canada,[1277 York Mills Rd (At Parkwoods Village Driv...,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[],At Parkwoods Village Drive,NaN
7,e-0-4fb699dd7bebbeb2a6c7badb-7,0,"[{'summary': 'This spot is popular', 'type': '...",4fb699dd7bebbeb2a6c7badb,Shoppers Drug Mart,51 Underhill Dr,43.745315,-79.325800,"[{'label': 'display', 'lat': 43.745315, 'lng':...",937,...,NaN,North York,ON,Canada,"[51 Underhill Dr (at Donwood Plaza), North Yor...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[],at Donwood Plaza,NaN
8,e-0-4b26adc3f964a5205f7f24e3-8,0,"[{'summary': 'This spot is popular', 'type': '...",4b26adc3f964a5205f7f24e3,Food Basics,1277 York Mills Rd,43.760549,-79.326045,"[{'label': 'display', 'lat': 43.76054899113784...",861,...,Parkwoods - Donalda,Toronto,ON,Canada,[1277 York Mills Rd (at Parkwoods Village Dr.)...,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",0,[],at Parkwoods Village Dr.,603662316
9,e-0-4b5f7253f964a520d7ba29e3-9,0,"[{'summary': 'This spot is popular', 'type': '...",4b5f7253f964a520d7ba29e3,Pizza Pizza,"1277 York Mills Road, Unit 9",43.760231,-79.325666,"[{'label': 'display', 'la

In [90]:
column_names = ['Name','Address', 'lat','lng'] 
ny_venues = pd.DataFrame(columns=column_names)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,referralId,count,items,id,name,address,lat,lng,labeledLatLngs,distance,...,neighborhood,city,state,country,formattedAddress,categories,count,groups,crossStreet,id
0,e-0-4b8991cbf964a520814232e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b8991cbf964a520814232e3,Allwyn's Bakery,81 Underhill drive,43.759840,-79.324719,"[{'label': 'display', 'lat': 43.75984035203157...",833,...,Parkwoods - Donalda,Toronto,ON,Canada,"[81 Underhill drive, Toronto ON M3A 1Z5, Canada]","[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",0,[],NaN,NaN
1,e-0-57e286f2498e43d84d92d34a-1,0,"[{'summary': 'This spot is popular', 'type': '...",57e286f2498e43d84d92d34a,Tim Hortons,215 Brookbanks,43.760668,-79.326368,"[{'label': 'display', 'lat': 43.76066827030228...",866,...,NaN,Toronto,ON,Canada,"[215 Brookbanks (York Miils Rd), Toronto ON M3...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],York Miils Rd,NaN
2,e-0-4e8d9dcdd5fbbbb6b3003c7b-2,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,...,NaN,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,600917367
3,e-0-4bafa285f964a5203a123ce3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4bafa285f964a5203a123ce3,Bruno's valu-mart,83 Underhill,43.746143,-79.324630,"[{'label': 'display', 'lat': 43.746143, 'lng':...",889,...,NaN,Don Mills,ON,Canada,"[83 Underhill (at Donwood Plaza), Don Mills ON...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[],at Donwood Plaza,NaN
4,e-0-4b845f3ff964a520733131e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b845f3ff964a520733131e3,High Street Fish & Chips,55 Underhill Dr.,43.745260,-79.324949,"[{'label': 'display', 'lat': 43.74525988387223...",967,...,NaN,Toronto,ON,Canada,"[55 Underhill Dr. (at Donwood Plaza), Toronto ...","[{'id': '4edd64a0c7ddd24ca188df1a', 'name': 'F...",0,[],at Donwood Plaza,NaN


In [92]:
ny_venues= nearby_venues[['name', 'address','lat','lng']]

In [93]:
ny_venues

,name,address,lat,lng
0,Allwyn's Bakery,81 Underhill drive,43.759840,-79.324719
1,Tim Hortons,215 Brookbanks,43.760668,-79.326368
2,Brookbanks Park,Toronto,43.751976,-79.332140
3,Bruno's valu-mart,83 Underhill,43.746143,-79.324630
4,High Street Fish & Chips,55 Underhill Dr.,43.745260,-79.324949
5,A&W,1277 York Mills Road,43.760643,-79.326865
6,Shoppers Drug Mart,1277 York Mills Rd,43.760857,-79.324961
7,Shoppers Drug Mart,51 Underhill Dr,43.745315,-79.325800
8,Food Basics,1277 York Mills Rd,43.760549,-79.326045
9,Pizza Pizza,"1277 York Mills Road, Unit 9",43.760231,-79.325666
